# Dataset 2022 Bereinigung und Normalisierung
## Spezialisiertes Modul für Springer/Immowelt/Immonet Dataset

### Ziel
Bereinigung und Normalisierung des aktuellen Datasets (2022) in ein standardisiertes Format für die gemeinsame Analyse.

### Input
- `data/raw/Dataset_2022.csv`
- `data/processed/berlin_plz_mapping.csv` (PLZ-zu-Bezirk-Mapping)

### Output
- `data/processed/dataset_2022_normalized.csv`

### Besonderheiten
- **PLZ-zu-Bezirk-Mapping erforderlich** (Dataset enthält nur PLZ, keine Bezirksnamen)
- Umfangreiche Spaltenstruktur mit vielen Features
- Deutsche Zahlenformate

### Standardisierte Ausgabespalten
- `price`: Normalisierter Preis (KALTMIETE in €)
- `size`: Normalisierte Größe (WOHNFLAECHE in m²)
- `district`: Berliner Bezirk (via PLZ-Mapping)
- `rooms`: Anzahl Zimmer (ZIMMER)
- `year`: Jahr des Datasets (2022)
- `dataset_id`: Eindeutige Dataset-Kennzeichnung (current)
- `source`: Datenquelle

---
**Teil der modularen Preprocessing-Pipeline**  
**Datum:** 4. Juli 2025  
**Version:** 1.0

## 1. Import Required Libraries

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)

print("Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print(f"Dataset: 2022 (Springer/Immowelt/Immonet)")

Bibliotheken erfolgreich importiert!
Pandas Version: 2.3.0
Dataset: 2022 (Springer/Immowelt/Immonet)


In [12]:
# ===================================================================
# ROBUST PLZ-TO-STRING CONVERSION FUNCTION
# ===================================================================
def convert_plz_to_string(plz_value):
    """
    Robust PLZ-to-string conversion function.
    Handles floats, ints, and strings properly.
    Returns clean string PLZ without .0 suffix, or None if invalid.
    
    Examples:
    - 10117.0 -> "10117"
    - 10117 -> "10117"
    - "10117" -> "10117"
    - nan -> None
    - "invalid" -> None
    """
    if pd.isna(plz_value):
        return None
    
    # Handle string inputs
    if isinstance(plz_value, str):
        plz_value = plz_value.strip()
        if plz_value.lower() in ['nan', 'none', '']:
            return None
        # Remove .0 suffix if present
        if plz_value.endswith('.0'):
            plz_value = plz_value[:-2]
        # Validate PLZ format (5 digits)
        if plz_value.isdigit() and len(plz_value) == 5:
            return plz_value
        return None
    
    # Handle numeric inputs (int or float)
    try:
        plz_int = int(plz_value)
        if 10000 <= plz_int <= 99999:  # Valid German PLZ range
            return str(plz_int)
        return None
    except (ValueError, TypeError):
        return None

# ===================================================================
# DEBUG LOGGING FUNCTION
# ===================================================================
def debug_plz_coverage(df, stage_name):
    """Debug function to track PLZ coverage at each stage"""
    total_rows = len(df)
    plz_coverage = df['plz'].notna().sum()
    plz_percentage = (plz_coverage / total_rows * 100) if total_rows > 0 else 0
    
    print(f"🔍 DEBUG - {stage_name}:")
    print(f"   Total rows: {total_rows:,}")
    print(f"   PLZ coverage: {plz_coverage:,} ({plz_percentage:.1f}%)")
    print(f"   PLZ missing: {total_rows - plz_coverage:,}")
    
    # Show PLZ data types
    if plz_coverage > 0:
        plz_types = df['plz'].dropna().apply(type).value_counts()
        type_dict = {}
        for type_key, count in plz_types.items():
            type_dict[str(type_key)] = count
        print(f"   PLZ data types: {type_dict}")
        # Show sample PLZ values
        sample_plz = df['plz'].dropna().head(3).tolist()
        print(f"   Sample PLZ: {sample_plz}")
    print()

print("✅ PLZ conversion and debug functions loaded!")
print("   • convert_plz_to_string(): Robust PLZ format conversion")
print("   • debug_plz_coverage(): Track PLZ coverage at each stage")

✅ PLZ conversion and debug functions loaded!
   • convert_plz_to_string(): Robust PLZ format conversion
   • debug_plz_coverage(): Track PLZ coverage at each stage


## 2. PLZ-zu-Bezirk-Mapping laden

In [2]:
# PLZ-zu-Bezirk-Mapping laden
print("=" * 60)
print("PLZ-ZU-BEZIRK-MAPPING LADEN")
print("=" * 60)

try:
    plz_mapping_df = pd.read_csv('data/processed/berlin_plz_mapping.csv')
    print(f"✅ PLZ-Mapping geladen: {len(plz_mapping_df)} Einträge")
    
    # Erstelle Dictionary für schnelles Lookup
    plz_to_district = dict(zip(plz_mapping_df['PLZ'], plz_mapping_df['Bezirk']))
    print(f"✅ PLZ-Dictionary erstellt: {len(plz_to_district)} Zuordnungen")
    
    # Zeige einige Beispiele
    print(f"\nBeispiele:")
    for plz, bezirk in list(plz_to_district.items())[:5]:
        print(f"  {plz} → {bezirk}")
        
    print(f"\nAbgedeckte Bezirke: {len(set(plz_to_district.values()))}")
    print(f"Bezirke: {sorted(set(plz_to_district.values()))}")
    
except FileNotFoundError:
    print("❌ FEHLER: PLZ-Mapping nicht gefunden!")
    print("Bitte stellen Sie sicher, dass 'data/processed/berlin_plz_mapping.csv' existiert.")
    raise

PLZ-ZU-BEZIRK-MAPPING LADEN
✅ PLZ-Mapping geladen: 181 Einträge
✅ PLZ-Dictionary erstellt: 181 Zuordnungen

Beispiele:
  10115 → Mitte
  10117 → Mitte
  10119 → Prenzlauer Berg
  10178 → Mitte
  10179 → Mitte

Abgedeckte Bezirke: 19
Bezirke: ['Charlottenburg', 'Friedrichshain', 'Kreuzberg', 'Lichtenberg', 'Marzahn-Hellersdorf', 'Mitte', 'Neukölln', 'Pankow', 'Prenzlauer Berg', 'Reinickendorf', 'Schöneberg', 'Spandau', 'Steglitz', 'Tempelhof', 'Tiergarten', 'Treptow-Köpenick', 'Wedding', 'Wilmersdorf', 'Zehlendorf']


## 3. Daten laden und erste Analyse

In [3]:
# Lade Dataset 2022
print("=" * 60)
print("DATASET 2022 LADEN UND ANALYSIEREN")
print("=" * 60)

# Lade Rohdaten
df_raw = pd.read_csv('data/raw/Dataset_2022.csv')
print(f"Dataset geladen: {df_raw.shape[0]:,} Zeilen, {df_raw.shape[1]} Spalten")

# Grundlegende Informationen
print(f"\nErste 10 Spalten: {list(df_raw.columns[:10])}")
print(f"Letzte 5 Spalten: {list(df_raw.columns[-5:])}")

# Kernfelder analysieren
core_fields = ['PLZ', 'KALTMIETE', 'WOHNFLAECHE', 'ZIMMER']
print(f"\n=== KERNFELDER ANALYSE ===")
for field in core_fields:
    if field in df_raw.columns:
        non_null = df_raw[field].notna().sum()
        print(f"{field}: {non_null}/{len(df_raw)} ({non_null/len(df_raw)*100:.1f}%) nicht-null")
    else:
        print(f"❌ {field}: Spalte nicht gefunden!")

# PLZ-Analyse
print(f"\n=== PLZ-ANALYSE ===")
unique_plz = df_raw['PLZ'].dropna().unique()
print(f"Einzigartige PLZ: {len(unique_plz)}")
print(f"PLZ-Beispiele: {sorted(unique_plz)[:10]}")

# Erste 5 Zeilen der Kernfelder
print(f"\nErste 5 Zeilen (Kernfelder):")
print(df_raw[core_fields].head())

DATASET 2022 LADEN UND ANALYSIEREN
Dataset geladen: 2,950 Zeilen, 75 Spalten

Erste 10 Spalten: ['ID', 'SORTE', 'PLZ', 'KALTMIETE', 'WARMMIETE', 'NEBENKOSTEN', 'KAUTION', 'HEIZUNGSKOSTEN', 'ZIMMER', 'PARKPLAETZE']
Letzte 5 Spalten: ['Stein', 'Marmor', 'Doppelboden', 'Terracotta', 'Sonstiges']

=== KERNFELDER ANALYSE ===
PLZ: 2950/2950 (100.0%) nicht-null
KALTMIETE: 2772/2950 (94.0%) nicht-null
WOHNFLAECHE: 2950/2950 (100.0%) nicht-null
ZIMMER: 2942/2950 (99.7%) nicht-null

=== PLZ-ANALYSE ===
Einzigartige PLZ: 183
PLZ-Beispiele: [np.int64(10115), np.int64(10117), np.int64(10119), np.int64(10178), np.int64(10179), np.int64(10243), np.int64(10245), np.int64(10247), np.int64(10249), np.int64(10315)]

Erste 5 Zeilen (Kernfelder):
     PLZ  KALTMIETE  WOHNFLAECHE  ZIMMER
0  13125     860.00        73.00     3.0
1  13125     450.28        48.84     2.0
2  13125     739.00        54.79     2.0
3  13125     899.00        74.49     3.0
4  13125     899.00        74.49     3.0


## 4. Spezifische Bereinigung Dataset 2022

In [4]:
# Spezifische Bereinigung für Dataset 2022
print("=" * 60)
print("SPEZIFISCHE BEREINIGUNG DATASET 2022")
print("=" * 60)

# Erstelle Arbeitskopie
df_clean = df_raw.copy()
print(f"Arbeitskopie erstellt: {len(df_clean)} Zeilen")

# === PREIS-BEREINIGUNG (KALTMIETE) ===
print("\n=== PREIS-BEREINIGUNG (KALTMIETE) ===")
print(f"KALTMIETE - Typ: {df_clean['KALTMIETE'].dtype}")
print(f"Nicht-null Werte: {df_clean['KALTMIETE'].notna().sum()}")

# Nur Zeilen mit gültigen Preisen behalten
df_clean = df_clean[df_clean['KALTMIETE'].notna()]
print(f"Nach Entfernung NaN-Preise: {len(df_clean)} Zeilen")

# Unrealistische Preise entfernen (< 100€ oder > 10000€) - Konsistent mit anderen Datasets
initial_count = len(df_clean)
df_clean = df_clean[(df_clean['KALTMIETE'] >= 100) & (df_clean['KALTMIETE'] <= 10000)]
removed_prices = initial_count - len(df_clean)
print(f"Gültige Preise nach Bereinigung: {len(df_clean)}/{initial_count} ({100*len(df_clean)/initial_count:.1f}%)")
print(f"Preisspanne: {df_clean['KALTMIETE'].min():.2f}€ - {df_clean['KALTMIETE'].max():.2f}€")
print(f"Entfernte unrealistische Preise: {removed_prices}")

# === GRÖSSEN-BEREINIGUNG (WOHNFLAECHE) ===
print("\n=== GRÖSSEN-BEREINIGUNG (WOHNFLAECHE) ===")
print(f"WOHNFLAECHE - Typ: {df_clean['WOHNFLAECHE'].dtype}")
print(f"Nicht-null Werte: {df_clean['WOHNFLAECHE'].notna().sum()}")

# Unrealistische Größen entfernen (< 10m² oder > 500m²) - Konsistent mit anderen Datasets
initial_count = len(df_clean)
df_clean = df_clean[(df_clean['WOHNFLAECHE'] >= 10) & (df_clean['WOHNFLAECHE'] <= 500)]
removed_sizes = initial_count - len(df_clean)
print(f"Gültige Größen nach Bereinigung: {len(df_clean)}/{initial_count} ({100*len(df_clean)/initial_count:.1f}%)")
print(f"Größenspanne: {df_clean['WOHNFLAECHE'].min():.1f}m² - {df_clean['WOHNFLAECHE'].max():.1f}m²")
print(f"Entfernte unrealistische Größen: {removed_sizes}")

# === PLZ-ZU-BEZIRK-ZUORDNUNG ===
print("\n=== PLZ-ZU-BEZIRK-ZUORDNUNG ===")
print(f"PLZ - Typ: {df_clean['PLZ'].dtype}")
print(f"Nicht-null PLZ: {df_clean['PLZ'].notna().sum()}")

# WICHTIG: Erweitere das PLZ-Mapping BEVOR die Zuordnung gemacht wird
print("\n=== ERWEITERUNG DES PLZ-MAPPINGS ===")
# Erweitere das PLZ-Dictionary um fehlende PLZ-Codes
additional_plz_mapping = {
    12627: 'Marzahn-Hellersdorf',
    12629: 'Marzahn-Hellersdorf',
    13593: 'Spandau',
    13595: 'Spandau',
    13597: 'Spandau',
    13599: 'Spandau',
    14052: 'Charlottenburg-Wilmersdorf',
    14055: 'Charlottenburg-Wilmersdorf',
    14057: 'Charlottenburg-Wilmersdorf',
    14059: 'Charlottenburg-Wilmersdorf',
    10315: 'Lichtenberg',
    10317: 'Lichtenberg',
    10318: 'Lichtenberg',
    10319: 'Lichtenberg',
    10365: 'Lichtenberg',
    10367: 'Lichtenberg',
    10369: 'Lichtenberg',
    13125: 'Pankow',
    13127: 'Pankow',
    13129: 'Pankow',
    13156: 'Pankow',
    13158: 'Pankow',
    13159: 'Pankow',
    13187: 'Pankow',
    13189: 'Pankow',
    12305: 'Tempelhof-Schöneberg',
    12307: 'Tempelhof-Schöneberg',
    12309: 'Tempelhof-Schöneberg',
    12347: 'Neukölln',
    12349: 'Neukölln',
    12351: 'Neukölln',
    12353: 'Neukölln',
    12355: 'Neukölln',
    12357: 'Neukölln',
    12359: 'Neukölln',
    12681: 'Marzahn-Hellersdorf',
    12683: 'Marzahn-Hellersdorf',
    12685: 'Marzahn-Hellersdorf',
    12687: 'Marzahn-Hellersdorf',
    12689: 'Marzahn-Hellersdorf',
    12679: 'Marzahn-Hellersdorf',
    13051: 'Pankow',
    13053: 'Pankow',
    13055: 'Pankow',
    13057: 'Pankow',
    13059: 'Pankow',
    13086: 'Pankow',
    13088: 'Pankow',
    13089: 'Pankow',
    13403: 'Reinickendorf',
    13405: 'Reinickendorf',
    13407: 'Reinickendorf',
    13409: 'Reinickendorf',
    13435: 'Reinickendorf',
    13437: 'Reinickendorf',
    13439: 'Reinickendorf',
    13465: 'Reinickendorf',
    13467: 'Reinickendorf',
    13469: 'Reinickendorf',
    13503: 'Reinickendorf',
    13505: 'Reinickendorf',
    13507: 'Reinickendorf',
    13509: 'Reinickendorf',
    13581: 'Spandau',
    13583: 'Spandau',
    13585: 'Spandau',
    13587: 'Spandau',
    13589: 'Spandau',
    13591: 'Spandau',
    14195: 'Steglitz-Zehlendorf',
    14197: 'Steglitz-Zehlendorf',
    14199: 'Steglitz-Zehlendorf',
    14163: 'Steglitz-Zehlendorf',
    14165: 'Steglitz-Zehlendorf',
    14167: 'Steglitz-Zehlendorf',
    14169: 'Steglitz-Zehlendorf',
    14129: 'Steglitz-Zehlendorf',
    14109: 'Steglitz-Zehlendorf',
}

# Erweitere das ursprüngliche PLZ-Dictionary
plz_to_district.update(additional_plz_mapping)
print(f"PLZ-Mapping erweitert: {len(plz_to_district)} Zuordnungen")

# Jetzt PLZ zu Bezirk zuordnen
df_clean['district'] = df_clean['PLZ'].map(plz_to_district)
successful_mappings = df_clean['district'].notna().sum()
print(f"Erfolgreiche PLZ-zu-Bezirk-Zuordnungen: {successful_mappings}/{len(df_clean)} ({100*successful_mappings/len(df_clean):.1f}%)")

# Zeige nicht zugeordnete PLZ
unmapped_plz = df_clean[df_clean['district'].isna()]['PLZ'].value_counts().head(10)
if len(unmapped_plz) > 0:
    unique_unmapped = df_clean[df_clean['district'].isna()]['PLZ'].nunique()
    print(f"\nVerbleibende nicht zugeordnete PLZ ({unique_unmapped} einzigartige):")
    for plz, count in unmapped_plz.items():
        print(f"  {plz}: {count} Einträge")

# Nur Zeilen mit gültigen Bezirken behalten
initial_count = len(df_clean)
df_clean = df_clean[df_clean['district'].notna()]
removed_no_district = initial_count - len(df_clean)
print(f"Entfernte Einträge ohne Bezirk: {removed_no_district}")

# === ZIMMER-BEREINIGUNG (ZIMMER) ===
print("\n=== ZIMMER-BEREINIGUNG (ZIMMER) ===")
print(f"ZIMMER - Typ: {df_clean['ZIMMER'].dtype}")
print(f"Nicht-null Werte: {df_clean['ZIMMER'].notna().sum()}")

# Unrealistische Zimmerzahlen entfernen
if len(df_clean) > 0:
    initial_count = len(df_clean)
    df_clean = df_clean[(df_clean['ZIMMER'] >= 1) & (df_clean['ZIMMER'] <= 10)]
    removed_rooms = initial_count - len(df_clean)
    print(f"Gültige Zimmerzahlen nach Bereinigung: {len(df_clean)}/{initial_count} ({100*len(df_clean)/initial_count:.1f}%)")
    if len(df_clean) > 0:
        print(f"Zimmerspanne: {df_clean['ZIMMER'].min():.1f} - {df_clean['ZIMMER'].max():.1f}")
    print(f"Entfernte unrealistische Zimmerzahlen: {removed_rooms}")

print(f"\n✅ Spezifische Bereinigung abgeschlossen")
print(f"Verbleibende Datensätze: {len(df_clean)} (Verlust: {len(df_raw) - len(df_clean)})")

SPEZIFISCHE BEREINIGUNG DATASET 2022
Arbeitskopie erstellt: 2950 Zeilen

=== PREIS-BEREINIGUNG (KALTMIETE) ===
KALTMIETE - Typ: float64
Nicht-null Werte: 2772
Nach Entfernung NaN-Preise: 2772 Zeilen
Gültige Preise nach Bereinigung: 2772/2772 (100.0%)
Preisspanne: 163.31€ - 3000.00€
Entfernte unrealistische Preise: 0

=== GRÖSSEN-BEREINIGUNG (WOHNFLAECHE) ===
WOHNFLAECHE - Typ: float64
Nicht-null Werte: 2772
Gültige Größen nach Bereinigung: 2772/2772 (100.0%)
Größenspanne: 13.0m² - 230.0m²
Entfernte unrealistische Größen: 0

=== PLZ-ZU-BEZIRK-ZUORDNUNG ===
PLZ - Typ: int64
Nicht-null PLZ: 2772

=== ERWEITERUNG DES PLZ-MAPPINGS ===
PLZ-Mapping erweitert: 188 Zuordnungen
Erfolgreiche PLZ-zu-Bezirk-Zuordnungen: 2681/2772 (96.7%)

Verbleibende nicht zugeordnete PLZ (10 einzigartige):
  12247: 16 Einträge
  12207: 16 Einträge
  13627: 12 Einträge
  12203: 11 Einträge
  12209: 9 Einträge
  12279: 7 Einträge
  12277: 7 Einträge
  12249: 6 Einträge
  10551: 4 Einträge
  12205: 3 Einträge
Entfer

## 5. Normalisierung in Standardformat

In [5]:
# Normalisierung in Standardformat
print("=" * 60)
print("NORMALISIERUNG IN STANDARDFORMAT")
print("=" * 60)

# Erstelle normalisiertes Dataset mit Standardspalten
df_normalized = pd.DataFrame()

if len(df_clean) > 0:
    # Standardspalten zuweisen
    df_normalized['price'] = df_clean['KALTMIETE'].astype('float64')
    df_normalized['size'] = df_clean['WOHNFLAECHE'].astype('float64')
    df_normalized['district'] = df_clean['district'].astype('string')
    df_normalized['rooms'] = df_clean['ZIMMER'].astype('float64')
    df_normalized['year'] = 2022
    df_normalized['dataset_id'] = 'current'
    df_normalized['source'] = 'Springer/Immowelt/Immonet'
    df_normalized['plz'] = df_clean['PLZ'].astype('string')

    # Zusätzliche wichtige Spalten aus Original-Dataset beibehalten
    if 'WARMMIETE' in df_clean.columns:
        df_normalized['warmmiete'] = df_clean['WARMMIETE']
    if 'NEBENKOSTEN' in df_clean.columns:
        df_normalized['nebenkosten'] = df_clean['NEBENKOSTEN']
    if 'KAUTION' in df_clean.columns:
        df_normalized['kaution'] = df_clean['KAUTION']
    if 'BAUJAHR' in df_clean.columns:
        df_normalized['baujahr'] = df_clean['BAUJAHR']
    if 'ZUSTAND' in df_clean.columns:
        df_normalized['zustand'] = df_clean['ZUSTAND']
    if 'ENERGIEEFFIZIENSKLASSE' in df_clean.columns:
        df_normalized['energieeffiziensklasse'] = df_clean['ENERGIEEFFIZIENSKLASSE']

    # Ausstattungsmerkmale (boolean)
    ausstattung_cols = ['möbliert', 'Balkon', 'Terrasse', 'Garten', 'Einbauküche', 
                       'Garage', 'Stellplatz', 'Personenaufzug', 'Keller']
    for col in ausstattung_cols:
        if col in df_clean.columns:
            df_normalized[f'ausstattung_{col.lower()}'] = df_clean[col]

print(f"Normalisiertes Dataset erstellt: {len(df_normalized)} Zeilen")
print(f"Standardspalten: {['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'plz']}")
print(f"Zusätzliche Spalten: {len(df_normalized.columns) - 8}")

# Datenqualität prüfen
print(f"\n=== DATENQUALITÄT NORMALISIERTES DATASET ===")
print(f"Zeilen mit Preis: {df_normalized['price'].notna().sum()}")
print(f"Zeilen mit Größe: {df_normalized['size'].notna().sum()}")
print(f"Zeilen mit Bezirk: {df_normalized['district'].notna().sum()}")
print(f"Zeilen mit Zimmeranzahl: {df_normalized['rooms'].notna().sum()}")

# Statistiken
if len(df_normalized) > 0:
    print(f"\n=== STATISTIKEN ===")
    print(f"Preis - Min: {df_normalized['price'].min():.2f}€, Max: {df_normalized['price'].max():.2f}€, Median: {df_normalized['price'].median():.2f}€")
    print(f"Größe - Min: {df_normalized['size'].min():.1f}m², Max: {df_normalized['size'].max():.1f}m², Median: {df_normalized['size'].median():.1f}m²")
    print(f"Zimmer - Min: {df_normalized['rooms'].min():.1f}, Max: {df_normalized['rooms'].max():.1f}, Median: {df_normalized['rooms'].median():.1f}")

    # Bezirksverteilung
    print(f"\n=== BEZIRKSVERTEILUNG ===")
    district_counts = df_normalized['district'].value_counts()
    print(f"Anzahl Bezirke: {len(district_counts)}")
    for district, count in district_counts.head(10).items():
        print(f"  {district}: {count} Einträge")

print(f"\n✅ Normalisierung abgeschlossen!")

NORMALISIERUNG IN STANDARDFORMAT
Normalisiertes Dataset erstellt: 2676 Zeilen
Standardspalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'plz']
Zusätzliche Spalten: 15

=== DATENQUALITÄT NORMALISIERTES DATASET ===
Zeilen mit Preis: 2676
Zeilen mit Größe: 2676
Zeilen mit Bezirk: 2676
Zeilen mit Zimmeranzahl: 2676

=== STATISTIKEN ===
Preis - Min: 180.00€, Max: 3000.00€, Median: 790.60€
Größe - Min: 13.0m², Max: 230.0m², Median: 65.1m²
Zimmer - Min: 1.0, Max: 5.0, Median: 2.0

=== BEZIRKSVERTEILUNG ===
Anzahl Bezirke: 21
  Spandau: 329 Einträge
  Pankow: 256 Einträge
  Reinickendorf: 247 Einträge
  Treptow-Köpenick: 243 Einträge
  Marzahn-Hellersdorf: 237 Einträge
  Lichtenberg: 151 Einträge
  Neukölln: 147 Einträge
  Friedrichshain: 113 Einträge
  Schöneberg: 107 Einträge
  Mitte: 99 Einträge

✅ Normalisierung abgeschlossen!


## 6. Export des normalisierten Datasets

In [6]:
# Export des normalisierten Datasets
print("=" * 60)
print("EXPORT NORMALISIERTES DATASET")
print("=" * 60)

# Ausgabedatei
output_file = 'data/processed/dataset_2022_normalized.csv'

# Export
df_normalized.to_csv(output_file, index=False)
print(f"✅ Normalisiertes Dataset exportiert: {output_file}")
print(f"Dateigröße: {len(df_normalized)} Zeilen x {len(df_normalized.columns)} Spalten")

# Validierung des Exports
test_load = pd.read_csv(output_file)
print(f"✅ Export-Validierung erfolgreich: {len(test_load)} Zeilen geladen")

# Zusammenfassung
print(f"\n=== ZUSAMMENFASSUNG DATASET 2022 ===")
print(f"Input: data/raw/Dataset_2022.csv ({len(df_raw)} Zeilen)")
print(f"Output: {output_file} ({len(df_normalized)} Zeilen)")
print(f"Datenverlust: {len(df_raw) - len(df_normalized)} Zeilen ({((len(df_raw) - len(df_normalized))/len(df_raw)*100):.1f}%)")
print(f"PLZ-zu-Bezirk-Mapping: {df_normalized['district'].notna().sum()}/{len(df_normalized)} ({df_normalized['district'].notna().sum()/len(df_normalized)*100:.1f}%) erfolgreich")
print(f"Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source")
print(f"Zusätzliche Spalten: {len(df_normalized.columns) - 7}")

print(f"\n🎯 DATASET 2022 BEREINIGUNG ABGESCHLOSSEN!")
print(f"Bereit für Kombination mit anderen normalisierten Datasets.")

EXPORT NORMALISIERTES DATASET
✅ Normalisiertes Dataset exportiert: data/processed/dataset_2022_normalized.csv
Dateigröße: 2676 Zeilen x 23 Spalten
✅ Export-Validierung erfolgreich: 2676 Zeilen geladen

=== ZUSAMMENFASSUNG DATASET 2022 ===
Input: data/raw/Dataset_2022.csv (2950 Zeilen)
Output: data/processed/dataset_2022_normalized.csv (2676 Zeilen)
Datenverlust: 274 Zeilen (9.3%)
PLZ-zu-Bezirk-Mapping: 2676/2676 (100.0%) erfolgreich
Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source
Zusätzliche Spalten: 16

🎯 DATASET 2022 BEREINIGUNG ABGESCHLOSSEN!
Bereit für Kombination mit anderen normalisierten Datasets.


## 7. Lade angereicherte Wohnlagendaten

In [7]:
print("="*60)
print("ANGEREICHERTE WOHNLAGENDATEN LADEN")
print("="*60)

enriched_data_path = 'data/raw/wohnlagen_enriched.csv'
try:
    enriched_df = pd.read_csv(enriched_data_path)
    print(f"✅ Angereicherte Daten geladen: {len(enriched_df):,} Zeilen, {len(enriched_df.columns)} Spalten")
except FileNotFoundError:
    print(f"❌ Datei nicht gefunden: {enriched_data_path}")

ANGEREICHERTE WOHNLAGENDATEN LADEN
✅ Angereicherte Daten geladen: 551,249 Zeilen, 11 Spalten
✅ Angereicherte Daten geladen: 551,249 Zeilen, 11 Spalten


## 8. Kombiniere Datasets mit Wohnlagendaten

In [13]:
print("="*60)
print("KOMBINIERE MIT WOHNLAGENDATEN - ENHANCED WITH PLZ FIXES")
print("="*60)

# Debug: Check original data sizes
print(f"Original df_normalized: {len(df_normalized):,} Zeilen")
print(f"Original enriched_df: {len(enriched_df):,} Zeilen")

# ===================================================================
# ROBUST PLZ CONVERSION BEFORE ENRICHMENT
# ===================================================================
print("\n🔧 ROBUST PLZ CONVERSION")
print("=" * 50)

# Debug PLZ status before conversion
debug_plz_coverage(df_normalized, "Vor PLZ-Konvertierung")

# Convert PLZ to string using robust function
print("Konvertiere PLZ zu String-Format...")
df_normalized['plz'] = df_normalized['plz'].apply(convert_plz_to_string)

# Debug PLZ status after conversion
debug_plz_coverage(df_normalized, "Nach PLZ-Konvertierung")

# ===================================================================
# PREPARE ENRICHMENT DATA WITH PLZ CONVERSION
# ===================================================================
print("\n📦 VORBEREITUNG ANREICHERUNGSDATEN")
print("=" * 50)

# Convert enriched_df PLZ to string as well
print("Konvertiere enriched_df PLZ zu String...")
enriched_df['plz'] = enriched_df['plz'].apply(convert_plz_to_string)

# Create a unique mapping of PLZ to avoid cartesian product
enriched_df_subset = enriched_df[['plz', 'wol', 'ortsteil_neu']].drop_duplicates(subset=['plz'])
print(f"Unique PLZ mappings: {len(enriched_df_subset):,} Zeilen")

# Debug: Check PLZ overlap
df_plz_unique = set(df_normalized['plz'].dropna().unique())
enriched_plz_unique = set(enriched_df_subset['plz'].dropna().unique())
overlap = df_plz_unique.intersection(enriched_plz_unique)
print(f"PLZ overlap: {len(overlap)} von {len(df_plz_unique)} PLZ im Dataset")

if len(overlap) > 0:
    print(f"✅ Gute PLZ-Übereinstimmung für Anreicherung")
else:
    print(f"⚠️ Keine PLZ-Übereinstimmung - prüfe PLZ-Formate")

# ===================================================================
# PERFORM ENRICHMENT WITH DATA PRESERVATION
# ===================================================================
print("\n🔄 ANREICHERUNG MIT DATENERHALTUNG")
print("=" * 50)

# Store original PLZ count for comparison
original_plz_count = df_normalized['plz'].notna().sum()
print(f"PLZ vor Anreicherung: {original_plz_count:,}")

# Perform the merge (LEFT JOIN to preserve all original data)
df_enriched = pd.merge(df_normalized, enriched_df_subset, how='left', on=['plz'])

# Debug PLZ status after enrichment
debug_plz_coverage(df_enriched, "Nach Anreicherung")

# Verify PLZ preservation
plz_after_enrichment = df_enriched['plz'].notna().sum()
print(f"PLZ nach Anreicherung: {plz_after_enrichment:,}")

if plz_after_enrichment != original_plz_count:
    print(f"⚠️ PLZ-Verlust erkannt: {original_plz_count - plz_after_enrichment} PLZ verloren!")
else:
    print(f"✅ PLZ vollständig erhalten!")

print(f"✅ Kombiniertes und angereichertes Dataset erstellt: {len(df_enriched):,} Zeilen")

# ===================================================================
# ENRICHMENT SUCCESS VALIDATION
# ===================================================================
print("\n✅ ANREICHERUNGS-ERFOLG VALIDIERUNG")
print("=" * 50)

# Check merge success
successful_enrichment = df_enriched['ortsteil_neu'].notna().sum()
enrichment_rate = (successful_enrichment / len(df_enriched)) * 100
print(f"Erfolgreiche Anreicherung: {successful_enrichment:,} von {len(df_enriched):,} Zeilen ({enrichment_rate:.1f}%)")

# Check WOL coverage
wol_success = df_enriched['wol'].notna().sum()
wol_rate = (wol_success / len(df_enriched)) * 100
print(f"WOL-Abdeckung: {wol_success:,} von {len(df_enriched):,} Zeilen ({wol_rate:.1f}%)")

# Final summary
print(f"\n📊 ANREICHERUNGS-ZUSAMMENFASSUNG:")
print(f"   • Original Dataset: {len(df_normalized):,} Zeilen")
print(f"   • Angereichert Dataset: {len(df_enriched):,} Zeilen")
print(f"   • PLZ-Erhaltung: {plz_after_enrichment:,}/{original_plz_count:,} ({'✅ Vollständig' if plz_after_enrichment == original_plz_count else '⚠️ Verlust'})")
print(f"   • Ortsteil-Anreicherung: {enrichment_rate:.1f}%")
print(f"   • WOL-Anreicherung: {wol_rate:.1f}%")

KOMBINIERE MIT WOHNLAGENDATEN - ENHANCED WITH PLZ FIXES
Original df_normalized: 2,676 Zeilen
Original enriched_df: 2,676 Zeilen

🔧 ROBUST PLZ CONVERSION
🔍 DEBUG - Vor PLZ-Konvertierung:
   Total rows: 2,676
   PLZ coverage: 2,676 (100.0%)
   PLZ missing: 0
   PLZ data types: {"<class 'str'>": 2676}
   Sample PLZ: ['13125', '13125', '13125']

Konvertiere PLZ zu String-Format...
🔍 DEBUG - Nach PLZ-Konvertierung:
   Total rows: 2,676
   PLZ coverage: 2,676 (100.0%)
   PLZ missing: 0
   PLZ data types: {"<class 'str'>": 2676}
   Sample PLZ: ['13125', '13125', '13125']


📦 VORBEREITUNG ANREICHERUNGSDATEN
Konvertiere enriched_df PLZ zu String...
Unique PLZ mappings: 173 Zeilen
PLZ overlap: 173 von 173 PLZ im Dataset
✅ Gute PLZ-Übereinstimmung für Anreicherung

🔄 ANREICHERUNG MIT DATENERHALTUNG
PLZ vor Anreicherung: 2,676
🔍 DEBUG - Nach Anreicherung:
   Total rows: 2,676
   PLZ coverage: 2,676 (100.0%)
   PLZ missing: 0
   PLZ data types: {"<class 'str'>": 2676}
   Sample PLZ: ['13125', '1312

## 9. Export des finalen angereicherten Datasets

In [14]:
print("="*60)
print("EXPORT FINALES ANGEREICHERTES DATASET - ENHANCED")
print("="*60)

# ===================================================================
# FINALIZE PLZ FORMAT BEFORE EXPORT
# ===================================================================
print("\n🔧 FINALIZE PLZ FORMAT")
print("=" * 50)

# Ensure PLZ is in string format for export
if 'plz' in df_enriched.columns:
    print("Finalisiere PLZ als String-Format...")
    df_enriched['plz'] = df_enriched['plz'].apply(convert_plz_to_string)
    
    # Final PLZ status
    final_plz_count = df_enriched['plz'].notna().sum()
    print(f"✅ Finale PLZ-Abdeckung: {final_plz_count:,}/{len(df_enriched):,} ({final_plz_count/len(df_enriched)*100:.1f}%)")
    
    # Show sample PLZ values to verify string format
    if final_plz_count > 0:
        sample_plz = df_enriched['plz'].dropna().head(5).tolist()
        print(f"✅ PLZ-Beispiele (String-Format): {sample_plz}")
        print(f"✅ PLZ-Datentyp: {type(sample_plz[0])}")

# ===================================================================
# EXPORT WITH PROPER DTYPES
# ===================================================================
print("\n📤 EXPORT MIT KORREKTEN DATENTYPEN")
print("=" * 50)

# Export
output_file_enriched = 'data/processed/dataset_2022_enriched.csv'
df_enriched.to_csv(output_file_enriched, index=False)

print(f"✅ Finales angereichertes Dataset exportiert: {output_file_enriched}")
print(f"   📊 Dateigröße: {len(df_enriched):,} Zeilen x {len(df_enriched.columns)} Spalten")

# ===================================================================
# EXPORT VALIDATION WITH CORRECT DTYPES
# ===================================================================
print("\n🔍 EXPORT-VALIDIERUNG MIT KORREKTEN DTYPES")
print("=" * 50)

# Validierung durch Wiedereinlesen mit korrekten dtypes
try:
    test_df_enriched = pd.read_csv(output_file_enriched, dtype={'plz': 'string'})
    print(f"✅ Export-Validierung erfolgreich: {len(test_df_enriched):,} Zeilen geladen")
    print(f"✅ PLZ-Datentyp beim Re-Import: {test_df_enriched['plz'].dtype}")
    
    # Verify PLZ coverage after reimport
    plz_final = test_df_enriched['plz'].notna().sum()
    ortsteil_final = test_df_enriched['ortsteil_neu'].notna().sum()
    print(f"📍 PLZ-Abdeckung: {plz_final:,}/{len(test_df_enriched):,} ({plz_final/len(test_df_enriched)*100:.1f}%)")
    print(f"🏘️  Ortsteil-Abdeckung: {ortsteil_final:,}/{len(test_df_enriched):,} ({ortsteil_final/len(test_df_enriched)*100:.1f}%)")
    
    # Check for .0 suffix issues
    if plz_final > 0:
        sample_plz_validation = test_df_enriched['plz'].dropna().head(3).tolist()
        print(f"✅ PLZ-Beispiele nach Re-Import: {sample_plz_validation}")
        has_dot_zero = any('.0' in str(plz) for plz in sample_plz_validation)
        if has_dot_zero:
            print("⚠️ WARNUNG: .0 Suffixe gefunden - PLZ-Konvertierung prüfen!")
        else:
            print("✅ Keine .0 Suffixe gefunden - PLZ-Format korrekt!")
    
except Exception as e:
    print(f"❌ Export-Validierung fehlgeschlagen: {e}")

# ===================================================================
# FINAL PROCESSING SUMMARY
# ===================================================================
print("\n📋 FINAL PROCESSING SUMMARY: DATASET 2022")
print("=" * 60)

# Count rows at each stage
original_count = len(df_raw)
normalized_count = len(df_normalized)
enriched_count = len(df_enriched)

print(f"🔄 DATENVERARBEITUNGSPIPELINE:")
print(f"   1. Raw Dataset (geladen):           {original_count:,} Zeilen")
print(f"   2. Nach Bereinigung & Normalisierung: {normalized_count:,} Zeilen")
print(f"   3. Nach Anreicherung:               {enriched_count:,} Zeilen")

# Calculate losses
normalization_loss = original_count - normalized_count
enrichment_loss = normalized_count - enriched_count
total_loss = original_count - enriched_count

print(f"\n📉 DATENVERLUST-ANALYSE:")
print(f"   • Verlust durch Bereinigung: {normalization_loss:,} Zeilen ({100*normalization_loss/original_count:.1f}%)")
print(f"   • Verlust durch Anreicherung: {enrichment_loss:,} Zeilen ({100*enrichment_loss/normalized_count:.1f}%)")
print(f"   • Gesamtverlust: {total_loss:,} Zeilen ({100*total_loss/original_count:.1f}%)")

# Enrichment statistics
if 'ortsteil_neu' in df_enriched.columns:
    enrichment_success = df_enriched['ortsteil_neu'].notna().sum()
    enrichment_rate = enrichment_success / len(df_enriched) * 100
    print(f"\n✅ ANREICHERUNGSSTATISTIKEN:")
    print(f"   • Erfolgreich angereichert: {enrichment_success:,} von {len(df_enriched):,} Zeilen")
    print(f"   • Anreicherungsrate: {enrichment_rate:.1f}%")
    
    # PLZ improvement
    plz_success = df_enriched['plz'].notna().sum()
    plz_rate = plz_success / len(df_enriched) * 100
    print(f"   • PLZ-Abdeckung: {plz_success:,} von {len(df_enriched):,} Zeilen ({plz_rate:.1f}%)")

print(f"\n🎯 QUALITÄTSSICHERUNG:")
print(f"   ✅ PLZ als String-Datentyp gespeichert")
print(f"   ✅ Kein PLZ-Verlust während Anreicherung")
print(f"   ✅ Debug-Logging für PLZ-Pipeline implementiert")
print(f"   ✅ Konsistente Datenverarbeitung")

print(f"\n🎉 DATASET 2022 PROCESSING ERFOLGREICH ABGESCHLOSSEN!")
print(f"    Ready for next pipeline step: 04_Combine_Datasets.ipynb")
print("=" * 60)

EXPORT FINALES ANGEREICHERTES DATASET - ENHANCED

🔧 FINALIZE PLZ FORMAT
Finalisiere PLZ als String-Format...
✅ Finale PLZ-Abdeckung: 2,676/2,676 (100.0%)
✅ PLZ-Beispiele (String-Format): ['13125', '13125', '13125', '13125', '13125']
✅ PLZ-Datentyp: <class 'str'>

📤 EXPORT MIT KORREKTEN DATENTYPEN
✅ Finales angereichertes Dataset exportiert: data/processed/dataset_2022_enriched.csv
   📊 Dateigröße: 2,676 Zeilen x 25 Spalten

🔍 EXPORT-VALIDIERUNG MIT KORREKTEN DTYPES
✅ Export-Validierung erfolgreich: 2,676 Zeilen geladen
✅ PLZ-Datentyp beim Re-Import: string
📍 PLZ-Abdeckung: 2,676/2,676 (100.0%)
🏘️  Ortsteil-Abdeckung: 2,676/2,676 (100.0%)
✅ PLZ-Beispiele nach Re-Import: ['13125', '13125', '13125']
✅ Keine .0 Suffixe gefunden - PLZ-Format korrekt!

📋 FINAL PROCESSING SUMMARY: DATASET 2022
🔄 DATENVERARBEITUNGSPIPELINE:
   1. Raw Dataset (geladen):           2,950 Zeilen
   2. Nach Bereinigung & Normalisierung: 2,676 Zeilen
   3. Nach Anreicherung:               2,676 Zeilen

📉 DATENVERLUS

In [10]:
# VALIDATION: Check PLZ data type and format
print("=" * 60)
print("PLZ VALIDATION: Dataset 2022")
print("=" * 60)

# Check both normalized and enriched datasets
datasets_to_check = [
    ('normalized', 'data/processed/dataset_2022_normalized.csv'),
    ('enriched', 'data/processed/dataset_2022_enriched.csv')
]

for dataset_name, file_path in datasets_to_check:
    print(f"\n🔍 Checking {dataset_name} dataset...")
    
    try:
        # Load without dtype constraint first to see current state
        df_check = pd.read_csv(file_path)
        
        print(f"✅ Dataset loaded: {len(df_check):,} rows")
        
        if 'plz' in df_check.columns:
            plz_count = df_check['plz'].notna().sum()
            plz_percentage = (plz_count / len(df_check)) * 100
            
            print(f"✅ PLZ coverage: {plz_count:,}/{len(df_check):,} ({plz_percentage:.1f}%)")
            print(f"✅ PLZ data type: {df_check['plz'].dtype}")
            
            # Check sample values
            if plz_count > 0:
                sample_plz = df_check['plz'].dropna().head(5).tolist()
                print(f"✅ Sample PLZ values: {sample_plz}")
                
                # Check for .0 suffixes
                sample_plz_str = [str(plz) for plz in sample_plz]
                has_dot_zero = any('.0' in plz_str for plz_str in sample_plz_str)
                
                if has_dot_zero:
                    print("⚠️ WARNING: .0 suffixes detected in PLZ values!")
                    print("   This indicates PLZ is stored as float, not string")
                    print("   This may cause issues in downstream processing")
                else:
                    print("✅ No .0 suffixes detected - PLZ format appears correct")
            
            # Check for PLZ data type issues
            if df_check['plz'].dtype in ['float64', 'int64']:
                print(f"⚠️ WARNING: PLZ stored as {df_check['plz'].dtype}")
                print("   Should be string type for proper processing")
                print("   This may cause issues in joins and mapping")
            elif df_check['plz'].dtype == 'object':
                print("✅ PLZ stored as object type (likely string)")
            else:
                print(f"✅ PLZ stored as {df_check['plz'].dtype}")
                
        else:
            print("❌ No PLZ column found!")
            
    except Exception as e:
        print(f"❌ Error loading {dataset_name} dataset: {e}")

print(f"\n🎯 VALIDATION SUMMARY:")
print(f"   Dataset 2022 appears to have good PLZ coverage (100%)")
print(f"   Need to verify PLZ data type consistency")
print(f"   If PLZ is stored as float/int, needs conversion to string")
print("=" * 60)

PLZ VALIDATION: Dataset 2022

🔍 Checking normalized dataset...
✅ Dataset loaded: 2,676 rows
✅ PLZ coverage: 2,676/2,676 (100.0%)
✅ PLZ data type: int64
✅ Sample PLZ values: [13125, 13125, 13125, 13125, 13125]
✅ No .0 suffixes detected - PLZ format appears correct
⚠️ WARNING: PLZ stored as int64
   Should be string type for proper processing
   This may cause issues in joins and mapping

🔍 Checking enriched dataset...
✅ Dataset loaded: 2,676 rows
✅ PLZ coverage: 2,676/2,676 (100.0%)
✅ PLZ data type: int64
✅ Sample PLZ values: [13125, 13125, 13125, 13125, 13125]
✅ No .0 suffixes detected - PLZ format appears correct
⚠️ WARNING: PLZ stored as int64
   Should be string type for proper processing
   This may cause issues in joins and mapping

🎯 VALIDATION SUMMARY:
   Dataset 2022 appears to have good PLZ coverage (100%)
   Need to verify PLZ data type consistency
   If PLZ is stored as float/int, needs conversion to string


In [11]:
# QUICK PLZ STATUS CHECK
print("QUICK PLZ STATUS CHECK - Dataset 2022")
print("=" * 50)

# Check enriched dataset
enriched_df = pd.read_csv('data/processed/dataset_2022_enriched.csv')
print(f"Enriched dataset: {len(enriched_df):,} rows")
print(f"PLZ column exists: {'plz' in enriched_df.columns}")

if 'plz' in enriched_df.columns:
    plz_count = enriched_df['plz'].notna().sum()
    print(f"PLZ coverage: {plz_count:,}/{len(enriched_df):,} ({plz_count/len(enriched_df)*100:.1f}%)")
    print(f"PLZ data type: {enriched_df['plz'].dtype}")
    
    # Check sample values
    sample_plz = enriched_df['plz'].dropna().head(3).tolist()
    print(f"Sample PLZ: {sample_plz}")
    
    # Check for .0 suffixes
    has_dot_zero = any('.0' in str(plz) for plz in sample_plz)
    print(f"Has .0 suffixes: {has_dot_zero}")
    
    if enriched_df['plz'].dtype in ['float64', 'int64']:
        print("⚠️ PLZ stored as numeric - needs string conversion!")
        needs_fix = True
    else:
        print("✅ PLZ appears to be stored correctly")
        needs_fix = False
        
    print(f"Needs PLZ fix: {needs_fix}")
else:
    print("❌ No PLZ column found")

QUICK PLZ STATUS CHECK - Dataset 2022
Enriched dataset: 2,676 rows
PLZ column exists: True
PLZ coverage: 2,676/2,676 (100.0%)
PLZ data type: int64
Sample PLZ: [13125, 13125, 13125]
Has .0 suffixes: False
⚠️ PLZ stored as numeric - needs string conversion!
Needs PLZ fix: True
